In [1]:
import subprocess
import pandas as pd

In [2]:
model_name = 'models/08jul_nested1'
model_data = 'models-data/data_30junB.dat'

In [5]:
p = subprocess.Popen(['C:/Program Files (x86)/biogeme-2.4/biogeme-2.4/biogeme.exe', model_name,model_data], shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
for line in p.stdout.readlines():
    print (line),
retval = p.wait()

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
biogeme 2.4 [Sun Oct 25 23:01:47 WEST 2015]
Michel Bierlaire, EPFL
-- Compiled by biogeme@hotmail.com on MINGW32_NT-6.1
See http://biogeme.epfl.ch
                    !! CFSQP is available !!
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
	"In every non-trivial program there is at least one bug."


[13:39:24]patFileNames.cc:52  models/08jul_nested1.par does not exist
[13:39:24]patFileNames.cc:56  Trying default.par instead
[13:39:24]patBiogeme.cc:134  Read default.par
 Opening file models-data/data_30junB.dat
 Data  file... line 500	Memory: 59 Kb
 Total obs.:   837
 Total memory: 99.8398 Kb
 Run time for data processing: 00:00
 Init loglike=-985.941
     gmax Iter   radius        f(x)     Status       rhok nFree
 +1.01e+000    1 1.00e+000 +9.8594093e+002 ****Converg  +1.22e+000 7  ++ P
 +1.01e+000    2 2.00e+000 +7.7813372e+002 ****Converg  +1.08e+000 7  ++ P
 +1.01e+000    3 4.00e+000 +7.5271787e+002 ****Converg  +1.04e+00

In [6]:
with open(model_name+'_res.mod','wb') as simmod:
    with open(model_name+'.res','rb') as res:
        searchlines = res.readlines()
        foundit = False
        for i, line in enumerate(searchlines):
            if foundit==False:
                if b'[SampleEnum]' in line:
                    print('Found it.')
                    foundit = True
                simmod.write(line)
            else:
                if line.decode("utf-8").find('0\r\n')==0:
                    print('Changed to 100.')
                    simmod.write(b'100\r\n')
                    foundit==False
                else:
                    simmod.write(line)

Found it.
Changed to 100.


In [7]:
p = subprocess.Popen(['C:/Program Files (x86)/biogeme-2.4/biogeme-2.4/biosim.exe', model_name+'_res',model_data], shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
for line in p.stdout.readlines():
    print (line),
retval = p.wait()

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
biogeme 2.4 [Sun Oct 25 23:01:47 WEST 2015]
Michel Bierlaire, EPFL
-- Compiled by biogeme@hotmail.com on MINGW32_NT-6.1
See http://biogeme.epfl.ch
                    !! CFSQP is available !!
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
	"In every non-trivial program there is at least one bug."


[13:41:57]patFileNames.cc:52  models/08jul_nested1_res.par does not exist
[13:41:57]patFileNames.cc:56  Trying default.par instead
[13:41:57]patBiogeme.cc:134  Read default.par
 Opening file models-data/data_30junB.dat
 Data  file... line 500	Memory: 59 Kb
 Total obs.:   837
 Total memory: 99.8398 Kb
 59%	[500/838] Est. term. time: 07/08/16 13:41:57 (in 00:00)
 BIOSIM Input file
 Model specification:	models/08jul_nested1_res.mod
 BIOSIM Output file
 Sample enumeration:	models/08jul_nested1_res.enu


In [8]:
simres = pd.read_csv(model_name+'_res.enu',sep='\t')
simres.head()

,Choice_Id,P_choice,V_A1_TAXI,V_A2_TRANSIT,V_A3_OTHER,V_A4_FREE,P_A1_TAXI,Residual_A1_TAXI,P_A2_TRANSIT,Residual_A2_TRANSIT,...,Simul91,Simul92,Simul93,Simul94,Simul95,Simul96,Simul97,Simul98,Simul99,Simul100
0,1,0.579924,0.313162,-0.263533,-0.285569,-9999.99,0.579924,0.420076,0.214037,-0.214037,...,2,2,1,2,1,1,2,1,2,1
1,1,0.579924,0.313162,-0.263533,-0.285569,-9999.99,0.579924,0.420076,0.214037,-0.214037,...,1,1,1,1,1,1,1,1,3,3
2,1,0.579924,0.313162,-0.263533,-0.285569,-9999.99,0.579924,0.420076,0.214037,-0.214037,...,2,2,1,2,1,1,3,1,3,1
3,1,0.579924,0.313162,-0.263533,-0.285569,-9999.99,0.579924,0.420076,0.214037,-0.214037,...,1,1,1,1,2,1,3,2,2,3
4,1,0.579924,0.313162,-0.263533,-0.285569,-9999.99,0.579924,0.420076,0.214037,-0.214037,...,2,2,1,2,1,1,1,1,2,1


In [9]:
simul_start = list(simres.columns).index('Simul1')
from collections import Counter
def count_correct(row):
    return list(row[simul_start:]).count(row.Choice_Id)
def simul_mode(row):
    return int(Counter(list(row[simul_start:])).most_common(1)[0][0])

In [10]:
simres['Accuracy'] = simres.apply(count_correct,axis=1)
simres['Mode_Prediction'] = simres.apply(simul_mode,axis=1)

In [11]:
simres['ModeAccurate'] = simres['Choice_Id']==simres['Mode_Prediction']

In [12]:
simres.groupby('Choice_Id')['Accuracy'].mean()

Choice_Id
1    57.891429
2    21.739496
3    23.169014
4    99.977974
Name: Accuracy, dtype: float64

In [13]:
simres.groupby('Choice_Id')['ModeAccurate'].mean()

Choice_Id
1    0.982857
2    0.084034
3    0.000000
4    1.000000
Name: ModeAccurate, dtype: float64

In [14]:
simres.groupby(['Choice_Id','Mode_Prediction']).size()

Choice_Id  Mode_Prediction
1          1                  344
           2                    6
2          1                  109
           2                   10
3          1                  138
           2                    4
4          4                  227
dtype: int64